In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.sys.path.append('../../evaluation/')
import pickle
import numpy as np
import pandas as pd
import methods
import metrics

data_path = '../../data/simulated_2d/test/'


In [2]:
# load data
data = pickle.load(open(data_path+'/input_data.pkl', 'rb'))
targets = pickle.load(open(data_path+'/target_data.pkl', 'rb'))
targets = [
    metrics.output_to_stats_2d(i) for i in targets
]

In [3]:
p_gmm_1 = []
for c,i in enumerate(data):
    if c%200==0:
        print(c)
    p = methods.predict_gmm(i, init_weights=False, bic_logic=False)
    p_gmm_1.append([p['mean'], p['covariance']])
errors_gmm_1 = metrics.get_errors(targets, p_gmm_1)

p_gmm_2 = []
for c,i in enumerate(data):
    if c%200==0:
        print(c)
    p = methods.predict_gmm(i, init_weights=False, bic_logic=True)
    p_gmm_2.append([p['mean'], p['covariance']])        
errors_gmm_2 = metrics.get_errors(targets, p_gmm_2)

p_gmm_3 = []
for c,i in enumerate(data):
    if c%200==0:
        print(c)
    p = methods.predict_gmm(i, init_weights=True, bic_logic=False)
    p_gmm_3.append([p['mean'], p['covariance']])        
errors_gmm_3 = metrics.get_errors(targets, p_gmm_3)

p_gmm_4 = []
for c,i in enumerate(data):
    if c%200==0:
        print(c)
    p = methods.predict_gmm(i, init_weights=True, bic_logic=True)
    p_gmm_4.append([p['mean'], p['covariance']])        
errors_gmm_4 = metrics.get_errors(targets, p_gmm_4)


0
200
400
600
800
0
200
400
600
800
0
200
400
600
800
0
200
400
600
800


In [7]:
def get_error_statistics(errors):
    df = pd.DataFrame({
        'mean_bd': [np.nanmean(i) for i in errors.values()],
        'median_bd': [np.quantile(i, 0.5) for i in errors.values()],
        'acc_bc_0.95': [len(np.where(np.exp(-i)>=0.95)[0])/len(i) for i in errors.values()],
    })
    return df


In [8]:
errors = {
    'GMM1': errors_gmm_1,
    'GMM2': errors_gmm_2,
    'GMM3': errors_gmm_3,
    'GMM4': errors_gmm_4,
}

df = get_error_statistics(errors)
df.index = errors.keys()
df


,mean_bd,median_bd,acc_bc_0.95
GMM1,0.027030,0.006738,0.877
GMM2,0.026984,0.006687,0.877
GMM3,0.021591,0.006187,0.895
GMM4,0.021358,0.006164,0.896


In [11]:
df.loc['GMM4', 'mean_bd']/df.loc['GMM1', 'mean_bd']-1

-0.2098397275350653

In [12]:
df.loc['GMM4', 'acc_bc_0.95']/df.loc['GMM1', 'acc_bc_0.95']-1


0.02166476624857472

In [9]:
with open('./errors_gmm_all.pkl', 'wb') as f:
    pickle.dump(errors, f)
    